# TRAINING A CHATBOT INTENT CLASSIFIER


## Importing the libraries

In [34]:
import nltk
import numpy as np
import random

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [35]:
# download the wordnet and stopwords corpus
nltk.download('wordnet') # wordnet is a lexical database for the English language
nltk.download('stopwords') # stopwords means words like 'the', 'a', 'an', 'is', 'are', etc.
nltk.download('punkt') # punkt means punctuations

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dagbo_b40tnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dagbo_b40tnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dagbo_b40tnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Importing the json file

In [36]:
# import intents file
import json

def load_intents():
    with open('intents.json') as file:
        intents = json.load(file)
    return intents

intents = load_intents()

## Preprocessing the data

In [37]:
# initialize lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer() # lemma is the root form of the word and it is very accurate than stemming


# loop through each sentence in the intent's patterns
def preprocess_intents(intents):
    # create empty lists for documents, classes and words
    documents = [] # documents means patterns
    classes = [] # classes means tags
    words = [] # words means vocabulary
    ignore_letters = ['!', '?', ',', '.'] # ignore these letters
    stop_words = set(stopwords.words('english')) # stop words are words like 'the', 'a', 'an', 'is', 'are', etc.


    for intent in intents['intents']:
        
        # debug for keyerror @ 'patterns'
        # print(intent['patterns'])
        for pattern in intent['patterns']:
            # tokenize each and every word in the sentence
            word = nltk.word_tokenize(pattern) 
            # lemmatize each word and convert into lowercase
            word = [lemmatizer.lemmatize(w.lower()) for w in word if w not in stop_words and w not in ignore_letters]
            # add word to the word list
            words.extend(word) # extend means add to the list and append means add to the end of the list
            # add word(s) to documents
            documents.append((word, intent['tag'])) #
            # add tags to our classes list
            if intent['tag'] not in classes: # if tag is not in classes list
                classes.append(intent['tag']) # then add it to the classes list
                
    # sort words and remove duplicates
    words = sorted(list(set(words)))
    # sort classes
    classes = sorted(list(set(classes)))
    return documents, classes, words

documents, classes, words = preprocess_intents(intents)


## save the preprocessed data

In [38]:
import pickle

#save to file using pickle to dump
pickle.dump(words, open ('words.pkl', 'wb'))#wb means writing into binariries
pickle.dump(classes, open ('classes.pkl', 'wb'))


## Create training and testing data

In [39]:

#represents the words as numerical values using bag_of_words
train = []
empty_output = [0] * len(classes)

for doc in documents:
    #create an empty list of bag of words
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns  else bag.append(0)
    
    output = list(empty_output)
    output[classes.index(doc[1])] = 1
    # Ensure the 'bag' has the same length as 'words'
    assert len(bag) == len(words), f"Bag length: {len(bag)}, expected: {len(words)}"
    
    # Ensure the 'output' has the same length as 'classes'
    assert len(output) == len(classes), f"Output length: {len(output)}, expected: {len(classes)}"
    train.append([bag, output])

In [40]:
# shuffle the features and make numpy array
random.shuffle(train)
# Before converting to a NumPy array, ensure all elements in 'train' have the same length
#assert all(len(item) == len(train[0]) for item in train), "Not all lists in 'train' have the same length."
train = np.array(train, dtype=object)

# create train and test lists. X - patterns, Y - intents
train_x = list(train[:,0])
train_y = list(train[:,1])



## Building the model using long short term memory (LSTM)

In [41]:
# import tensorflow libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay # learning rate decay over each update

# Create a learning rate schedule
lr_schedule = ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)

# Build the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu')) # input layer
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu')) # hidden layer
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax')) # output layer

# Compile the model
sgd = SGD(learning_rate=lr_schedule, momentum=0.9, nesterov=True) # stochastic gradient descent
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])



## Training and saving the model

In [42]:
# Training and saving the model
solentBot = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
loss, accuracy = model.evaluate(np.array(train_x), np.array(train_y))
print(f"Loss: {loss}, accuracy: {accuracy}")
#model.save('solentBot_model.h5', solentBot)

Epoch 1/200
57/57 [==============================] - 4s 2ms/step - loss: 4.3868 - accuracy: 0.0071
Epoch 2/200
57/57 [==============================] - 0s 3ms/step - loss: 4.3605 - accuracy: 0.0283
Epoch 3/200
57/57 [==============================] - 0s 2ms/step - loss: 4.3279 - accuracy: 0.0495
Epoch 4/200
57/57 [==============================] - 0s 2ms/step - loss: 4.2936 - accuracy: 0.0424
Epoch 5/200
57/57 [==============================] - 0s 2ms/step - loss: 4.2551 - accuracy: 0.0742
Epoch 6/200
57/57 [==============================] - 0s 2ms/step - loss: 4.2218 - accuracy: 0.0495
Epoch 7/200
57/57 [==============================] - 0s 3ms/step - loss: 4.1784 - accuracy: 0.0530
Epoch 8/200
57/57 [==============================] - 0s 2ms/step - loss: 4.0808 - accuracy: 0.0919
Epoch 9/200
57/57 [==============================] - 0s 3ms/step - loss: 4.0515 - accuracy: 0.0813
Epoch 10/200
57/57 [==============================] - 0s 2ms/step - loss: 3.9298 - accuracy: 0.0919
Epoch 11/

In [43]:
# save the model to disk
model.save('solentBot_model.h5', solentBot)
print("model created, saved and trained")

model created, saved and trained


In [44]:
# evaluate the model to check for confidence score of the predictions
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [lemmatizer.lemmatize(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
                
    return np.array(bag)

def predict_class(s, model):
    bow = bag_of_words(s, words)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
        
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
            
    return result



In [45]:
# chatbot response
def chatbot_response(text):
    ints = predict_class(text, model)
    res = get_response(ints, intents)
    return res



In [46]:
# create GUI
import tkinter
from tkinter import *

def send():
    msg = EntryBox.get("1.0", 'end-1c').strip()
    EntryBox.delete("0.0", END)
    
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12))
        
        res = chatbot_response(msg)
        ChatLog.insert(END, "SolentBot: " + res + '\n\n')
        
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
base = Tk()
base.title("SolentBot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

# create chat window

ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

# bind scrollbar to chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")

ChatLog['yscrollcommand'] = scrollbar.set

# create button to send message
SendButton = Button(base, font=("Verdana", 12, 'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b", fg='#ffffff',
                    command=send)

# create the box to enter message
EntryBox = Text(base, bd=0, bg="white", width="29", height="5", font="Arial")

# place all components on the screen
scrollbar.place(x=376, y=6, height=386)

ChatLog.place(x=6, y=6, height=386, width=370)

EntryBox.place(x=128, y=401, height=90, width=265)

SendButton.place(x=6, y=401, height=90)
    
    
base.mainloop()



1/1 [==============================] - 0s 61ms/step


In [47]:
# test the confidence score of the predictions and show the percentage of the confidence score  just by typing a tag
print(predict_class('solent', model))
print(predict_class('courses', model))


1/1 [==============================] - 0s 25ms/step
[{'intent': 'name', 'probability': '0.51279646'}]
1/1 [==============================] - 0s 31ms/step
[{'intent': 'course_information', 'probability': '0.9935452'}]
